In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text


In [ ]:
import pandas as pd
df=pd.read_csv('spam.csv')
df.head(5)


In [ ]:
df.groupby('Category').describe()

In [ ]:
df.groupby('Category').value_counts()

In [ ]:
df['Category'].value_counts()

In [ ]:
df_spam=df[df['Category']=='spam']
df_spam.shape

In [ ]:
df_ham=df[df['Category']=='ham']
df_ham.shape

In [ ]:
df_ham.sample(20)

In [ ]:
df_ham_dpwnload=df_ham.sample(df_spam.shape[0])
df_ham_dpwnload.shape

In [ ]:
df_balanced=pd.concat([df_spam,df_ham_dpwnload])
df_balanced.shape

In [ ]:
df_balanced['Category'].value_counts()

In [ ]:
df_balanced.sample(5)

In [ ]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0 )
df_balanced.sample(5)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train , y_test=train_test_split(df_balanced['Massage'],df_balanced['spam'],stratify=df_balanced['spam'])


In [ ]:
X_train.heas(4)

In [ ]:

encoder_url='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
preprocess='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_process=hub.KerasLayer(preprocess) 
bert_encoder=hub.KerasLayer(encoder_url)

In [ ]:
def get_sentence_embeding(sentences):
    preprocessd_text=bert_process(sentences)
    bert_encoder(preprocessd_text)['pooled_output']
    get_sentence_embeding(["hello 500$, ...,","hi there,..."])

In [ ]:
e=get_sentence_embeding(['banana','mango','elon musk'])

In [ ]:
e

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[0],e[1]])

In [ ]:
#bert Layers
text_input=tf.keras.layers.Input(shape=(),dtype=tf.string,name='text')
preprocessed_text=bert_process(text_input)
outputs=bert_encoder(preprocessed_text)

#NN Layers
l=tf.keras.layers.Dropout(0,1,name='dropout')(outputs['pooled_output'])
l=tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)

#Contruct final model

model=tf.keras.Model(inputs=[text_input],outputs=[l])
model.summary()



In [ ]:
METRICS=[tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='percision'),
        tf.keras.metrics.Recall(name='recall')]

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=METRICS)


In [ ]:
model.fit(X_train,y_train,epochs=10)

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
y_predicted=model.predict(X_test)
y_predicted=y_predicted.flatten()

In [ ]:
import numpy as np
y_predicted=np.where(y_predicted>0.5,1,0)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_test,y_predicted)


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.headmap(cm,annot=True,fmt='d')
plt.xlabel('precision')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test,y_predicted))